# Contents:
1. Read images

### Import relevant libraries

In [14]:
import tensorflow as tf
from keras import layers
import matplotlib.pyplot as plt
from keras.applications import VGG16
from keras.models import Model
from keras.layers import Dense, Flatten

### Read from Directory

In [15]:
image_size = 256
batch_size = 32

In [16]:
dataset = tf.keras.utils.image_dataset_from_directory(
    "potato_dataset",
    labels='inferred',
    label_mode = 'int',
    batch_size = batch_size,
    image_size = (image_size,image_size),
    shuffle = True,
    seed= 42
)

Found 3302 files belonging to 3 classes.


In [17]:
class_names = dataset.class_names
class_names

['early_blight', 'late_blight', 'normal']

### Check if the data is balanced

In [18]:
# Create a dictionary to store class counts
class_counts = {}

# Loop through the dataset and count elements in each class
for images, labels in dataset:
    for label in labels.numpy():
        if label not in class_counts:
            class_counts[label] = 0
        class_counts[label] += 1

# Print the counts for each class
for label, count in class_counts.items():
    print(f'Class {label}: {count}')

min_count = min(class_counts)

Class 1: 1149
Class 0: 1133
Class 2: 1020


The data is distributed in acceptable range

### Split the data

In [19]:
def split_data(dataset, train_split = 0.8, val_split = 0.1, shuffle = True, shuffle_size= 10000):
    train_count = int(len(dataset)*train_split)
    val_count = int(len(dataset)*val_split)
    
    if shuffle:
        dataset = dataset.shuffle(shuffle_size, seed = 42)

    train_dataset = dataset.take(train_count)
    val_test_dataset = dataset.skip(train_count)

    validation_dataset = val_test_dataset.take(val_count)
    test_dataset = val_test_dataset.skip(val_count)

    return train_dataset, validation_dataset, test_dataset

In [20]:
train, validation, test = split_data(dataset)
print(f"train size = {len(train)}")
print(f"validation size = {len(validation)}")
print(f"test size = {len(test)}")

train size = 83
validation size = 10
test size = 11


### Cache and prefetch the data

In [21]:
train = train.cache().shuffle(10000).prefetch(buffer_size = tf.data.AUTOTUNE)
validation = validation.cache().shuffle(10000).prefetch(buffer_size = tf.data.AUTOTUNE)
test = test.cache().shuffle(10000).prefetch(buffer_size = tf.data.AUTOTUNE)

### Produce layers for model

In [22]:
resize_rescale_layer = tf.keras.Sequential([
    layers.Resizing(image_size,image_size),
    layers.Rescaling(1.0/255)
])

In [23]:
data_augmentation_layer = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.3),
    layers.RandomZoom(0.2)
])

### Model


In [24]:
input_shape = (image_size, image_size, 3)

def conv2D_layer(filter):
    return tf.keras.layers.Conv2D(
        filters=filter,
        kernel_size=(3,3),
        padding="valid",
        activation="relu"
    )

# Define input layer
input_layer = tf.keras.Input(shape=input_shape)

# Preprocessing layers
x = resize_rescale_layer(input_layer)
x = data_augmentation_layer(x)

# Convolutional layers
x = conv2D_layer(128)(x)
x = layers.MaxPool2D((2, 2))(x)
x = conv2D_layer(64)(x)
x = layers.MaxPool2D((2, 2))(x)
x = conv2D_layer(64)(x)
x = layers.MaxPool2D((2, 2))(x)
x = conv2D_layer(64)(x)
x = layers.MaxPool2D((2, 2))(x)
x = conv2D_layer(32)(x)
x = layers.MaxPool2D((2, 2))(x)

# Flatten layer
x = layers.Flatten()(x)

# Dense layers
x = layers.Dense(256, activation="relu")(x)
output_layer = layers.Dense(3, activation="softmax")(x)

# Define model
model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

# Build model
model.build(input_shape=input_shape)

In [25]:
model.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_2 (Sequential)       │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_3 (Sequential)       │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 254, 254, 128)  │         3,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 127, 127, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 125, 125, 64)   │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 60, 60, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 28, 28, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 12, 12, 32)     │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 6, 6, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 1152)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │           771 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 465,635 (1.78 MB)

 Trainable params: 465,635 (1.78 MB)

 Non-trainable params: 0 (0.00 B)

In [26]:
model.compile(
    optimizer='adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = False),
    metrics = ['accuracy']
)

EPOCHS = 10

history = model.fit(
    train,
    epochs= EPOCHS,
    batch_size=batch_size,
    verbose =1, 
    validation_data=validation
)

Epoch 1/10


83/83 ━━━━━━━━━━━━━━━━━━━━ 303s 4s/step - accuracy: 0.3672 - loss: 1.0948 - val_accuracy: 0.5312 - val_loss: 1.0137
Epoch 2/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 296s 4s/step - accuracy: 0.4999 - loss: 0.9849 - val_accuracy: 0.6594 - val_loss: 0.8570
Epoch 3/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 314s 4s/step - accuracy: 0.6593 - loss: 0.8223 - val_accuracy: 0.7188 - val_loss: 0.6340
Epoch 4/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 296s 4s/step - accuracy: 0.7309 - loss: 0.6331 - val_accuracy: 0.7906 - val_loss: 0.5326
Epoch 5/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 365s 4s/step - accuracy: 0.7538 - loss: 0.5759 - val_accuracy: 0.9062 - val_loss: 0.2842
Epoch 6/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 489s 6s/step - accuracy: 0.8826 - loss: 0.2934 - val_accuracy: 0.9312 - val_loss: 0.2101
Epoch 7/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 553s 7s/step - accuracy: 0.9133 - loss: 0.2542 - val_accuracy: 0.9156 - val_loss: 0.2552
Epoch 8/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 456s 5s/step - accuracy: 0.9315 - loss: 0.2144 - val_accuracy: 0.9594 - val_loss: 0.

In [30]:
history = model.fit(
    train,
    epochs= EPOCHS,
    batch_size=batch_size,
    verbose =1, 
    validation_data=validation
)
model.evaluate(test)
model.save("./weights/model_2.keras")

Epoch 1/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 641s 8s/step - accuracy: 0.9353 - loss: 0.1856 - val_accuracy: 0.9719 - val_loss: 0.1007
Epoch 2/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 636s 8s/step - accuracy: 0.9505 - loss: 0.1366 - val_accuracy: 0.9719 - val_loss: 0.0753
Epoch 3/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 644s 8s/step - accuracy: 0.9639 - loss: 0.0953 - val_accuracy: 0.9875 - val_loss: 0.0560
Epoch 4/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 639s 8s/step - accuracy: 0.9625 - loss: 0.1170 - val_accuracy: 0.9625 - val_loss: 0.0708
Epoch 5/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 650s 8s/step - accuracy: 0.9651 - loss: 0.1026 - val_accuracy: 0.9750 - val_loss: 0.0705
Epoch 6/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 650s 8s/step - accuracy: 0.9710 - loss: 0.1000 - val_accuracy: 0.9844 - val_loss: 0.0458
Epoch 7/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 645s 8s/step - accuracy: 0.9736 - loss: 0.0853 - val_accuracy: 0.9812 - val_loss: 0.0555
Epoch 8/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 644s 8s/step - accuracy: 0.9657 - loss: 0.0986 - val_accuracy: 0.9688 - v

In [27]:
model.evaluate(test)

11/11 ━━━━━━━━━━━━━━━━━━━━ 31s 1s/step - accuracy: 0.9685 - loss: 0.1465


[0.1649043709039688, 0.9545454382896423]

### Visualization of training

In [28]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

loss= history.history['loss']
val_loss = history.history['val_loss']

### Save the model

In [29]:
model.save("./weights/model_1.keras")